In [ ]:
#| default_exp mcp_tools
#| test: false

In [ ]:
#| export
#| eval: false

import logging, itertools, json, re
from functools import lru_cache
from dataclasses import dataclass, field
from typing import List, Dict, Any
from DataTalks.api_clients import Mode as Route_Mode
import httpx, yaml
from fastmcp import FastMCP  
from pydantic import ValidationError 


from DataTalks.parking_tools import (
    nearest_parking_ids,
    parking_status,
    facility_prediction,
)

from DataTalks.api_clients import (
    Point,
    osrm_car,
    osrm_walk,
    osrm_bike,
    digitransit_pt,
    Route
)

from DataTalks.api_clients import Point
from DataTalks.fastmcp_tools import mcp
from DataTalks.api_clients import Point
import httpx, pydantic

In [ ]:
#| export
#| eval: false

USER_AGENT = "ArenaBuddy/0.1 ( datatalks@example.com )"

In [ ]:
#| export
#| eval: false

@mcp.tool(
    name="geocode",
    description="""
    Convert a human-readable place or address into geographical coordinates
    (lat, lon) using the OpenStreetMap Nominatim service.
    """
)
async def geocode(query: str,
                  *,
                  countrycodes: str | None = None,
                  lang: str = "en,fi",
                  limit: int = 1) -> Point:
    """Return the first OpenStreetMap match for *query* (Nominatim)."""
    params = {
        "q": query,
        "format": "jsonv2",         
        "limit": limit,
        "accept-language": lang,
    }
    if countrycodes:
        params["countrycodes"] = countrycodes      

    async with httpx.AsyncClient(
        timeout=10,
        headers={"User-Agent": USER_AGENT}
    ) as cx:
        r = await cx.get(
            "https://nominatim.openstreetmap.org/search",
            params=params,
        )
    r.raise_for_status()
    hits = r.json()
    if not hits:
        raise LookupError(f"No location found for '{query}'")
    j = hits[0]
    return Point(lat=float(j["lat"]), lon=float(j["lon"]))

In [ ]:
#| export
#| eval: false

@mcp.tool(
    name="mount_openapi",
    description=(
        "Mount an OpenAPI or Swagger spec so its operations become callable "
        "tools. Args: name, spec_url, base_url (optional), http_timeout (s)."
    ),
)
async def mount_openapi(
    *,
    name: str,
    spec_path: str,
    base_url: str | None = None,
    http_timeout: float = 10.0,
    mcp_root: object | None = None,     
) -> List[str]:
    """
    Downloads `spec_url`.

    Returns the list of newly created **tool names**.
    Raises ToolError if the spec is invalid or an upstream URL cannot be
    inferred – the calling agent can catch this and try another API.
    """
    with open(spec_path, "r", encoding="utf-8") as fp:
        spec = json.load(fp)

    # ── 4. build HTTP client & sub-server ─────────────────────────────
    client = httpx.AsyncClient(base_url=base_url, timeout=http_timeout)
    try:
        sub = FastMCP.from_openapi(
            openapi_spec=spec,
            client=client,
            name=name,
            all_routes_as_tools=True,
        )
    except Exception as e:
        raise Exception(f"Failed to build FastMCP server: {e}") from e

    # flag for UI (does the spec declare any securitySchemes?)
    sub.requires_auth = bool(spec.get("components", {}).get("securitySchemes"))
    
    # ── 5. mount on the root router and return tool names ─────────────
    mcp.mount(name, sub)    
 
    
    return [t.name for t in (await sub.get_tools()).values()]